## 3. We now review k-fold cross-validation.

### (a) Explain how k-fold cross-validation is implemented.


- K folds cross validation is implemented by randomly dividing the data into k groups of roughly equal size. at first fold 1 is treated as val. the remaining folds are fitted and MSE is computed on the help out fold.  then the held out fold is shifted until all folds have been iterated over. Then you comput the average MSE acaross folds

### (b) What are the advantages and disadvantages of k-fold crossvalidation relative to:

#### i. The validation set approach?


- the validation estimate of test error can be highly variable. 
- only a subset of obs are used to fit model. statistical models perform worse with fewer observations
- con of k-folds is that is more complex 

#### ii. LOOCV?

- Main advantage of k-fold compared to LOOCV is that it is computationaly less intensive and takes less time to run
- also k-folds reduces variance
- Main con of k-fold compared to LOOCV is that it increases bias 

## 5. In Chapter 4, we used logistic regression to predict the probability of default using income and balance on the Default data set. We will now estimate the test error of this logistic regression model using the validation set approach. Do not forget to set a random seed before beginning your analysis. 

___Imports___

In [67]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA, QuadraticDiscriminantAnalysis as QDA
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score

import statsmodels.api as sm

from ISLP import load_data, confusion_table
from ISLP.models import ModelSpec as MS, summarize, contrast

In [68]:
def return_error(preds,y_test):
    TN = confusion_table(preds, y_test)[0][0]
    FP = confusion_table(preds, y_test)[0][1]
    FN = confusion_table(preds, y_test)[1][0]
    TP = confusion_table(preds, y_test)[1][1]

    error = (FP + FN) / (TN + FP + FN + TP)

    return error

### (a) Fit a logistic regression model that uses income and balance to predict default.

 ### (b) Using the validation set approach, estimate the test error of this model. In order to do this, you must perform the following steps:

 #### i. Split the sample set into a training set and a validation set. 

In [69]:
default = load_data('Default')
default_train, default_test = train_test_split(
    default,
    test_size = .2,
    random_state=27,
    shuffle= True
)

X_train = default_train.drop(['default', 'student'], axis = 1)
X_test = default_test.drop(['default', 'student'], axis = 1)
y_train = (default_train['default'] == 'Yes').astype(int)
y_test = (default_test['default'] == 'Yes').astype(int)

#### ii. Fit a multiple logistic regression model using only the training observations. 

In [81]:
glm = sm.GLM(y_train, X_train, family=sm.families.Binomial())
results = glm.fit()

#### iii. Obtain a prediction of default status for each individual in the validation set by computing the posterior probability of default for that individual, and classifying the individual to the default category if the posterior probability is greater than 0.5. 

In [71]:
# Making predictions on the test set
probs = results.predict(X_test)

# Assigning labels based on probability threshold
preds = np.array([0] * len(X_test))
preds[probs > 0.5] = 1

#### iv. Compute the validation set error, which is the fraction of the observations in the validation set that are misclassified. 

In [72]:
return_error(preds,y_test)

0.035

### (c) Repeat the process in (b) three times, using three different splits of the observations into a training set and a validation set. Comment on the results obtained

In [112]:
error_list = []

for random_state in range(1, 4):
    default_train, default_test = train_test_split(
        default,
        test_size=.2,
        random_state=random_state,
        shuffle=True
    )

    X_train = default_train.drop(['default', 'student'], axis=1)
    X_test = default_test.drop(['default', 'student'], axis=1)
    y_train = (default_train['default'] == 'Yes').astype(int)
    y_test = (default_test['default'] == 'Yes').astype(int)

    glm = sm.GLM(y_train, X_train, family=sm.families.Binomial())
    results = glm.fit()

    # Making predictions on the test set
    probs = results.predict(X_test)

    # Assigning labels based on probability threshold
    preds = np.array([0] * len(X_test))
    preds[probs > 0.5] = 1

    error = return_error(preds, y_test)
    error_list.append(error)

print(error_list)


[0.0295, 0.0285, 0.03]


- they are all basically the same...

### (d) Now consider a logistic regression model that predicts the probability of default using income, balance, and a dummy variable for student. Estimate the test error for this model using the validation set approach. Comment on whether or not including a dummy variable for student leads to a reduction in the test error rate. 

In [74]:
default.head()

,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138947
3,No,No,529.250605,35704.493935
4,No,No,785.655883,38463.495879


In [75]:
default_train, default_test = train_test_split(
    default,
    test_size = .2,
    random_state=3,
    shuffle= True
)

default_train['student']= (default_train['student'] == 'Yes').astype(int)
default_test['student']= (default_test['student'] == 'Yes').astype(int)

In [76]:
X_train = default_train.drop('default', axis = 1)
X_test = default_test.drop('default', axis = 1)
y_train = (default_train['default'] == 'Yes').astype(int)
y_test = (default_test['default'] == 'Yes').astype(int)


In [79]:

glm = sm.GLM(y_train, X_train, family=sm.families.Binomial())
results = glm.fit()

# Making predictions on the test set
probs = results.predict(X_test)

# Assigning labels based on probability threshold
preds = np.array([0] * len(X_test))
preds[probs > 0.5] = 1

return_error(preds,y_test)

0.033

- it is slightly worse

## 6. We continue to consider the use of a logistic regression model to predict the probability of default using income and balance on the Default data set. In particular, we will now compute estimates for the standard errors of the income and balance logistic regression coefficients in two different ways: (1) using the bootstrap, and (2) using the standard formula for computing the standard errors in the sm.GLM() function. Do not forget to set a random seed before beginning your analysis. 

In [80]:
import random
random.seed(27)

### (a) Using the summarize() and sm.GLM() functions, determine the estimated standard errors for the coefficients associated with income and balance in a multiple logistic regression model that uses both predictors. 

In [82]:
default_train, default_test = train_test_split(
    default,
    test_size = .2,
    random_state = 2,
    shuffle = True
)

X_train = default_train.drop(['default', 'student'], axis = 1)
X_test = default_test.drop(['default', 'student'], axis = 1)
y_train = (default_train['default'] == 'Yes').astype(int)
y_test = (default_test['default'] == 'Yes').astype(int)

glm = sm.GLM(y_train, X_train, family=sm.families.Binomial())
results = glm.fit()

# Summarizing results
print(results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                default   No. Observations:                 8000
Model:                            GLM   Df Residuals:                     7998
Model Family:                Binomial   Df Model:                            1
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1415.8
Date:                Mon, 26 Feb 2024   Deviance:                       2831.6
Time:                        20:42:08   Pearson chi2:                 2.79e+04
No. Iterations:                     7   Pseudo R-squ. (CS):           -0.05532
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
balance        0.0005    7.8e-05      5.786      0.0

- balance std err: 7.8e-05 
- income std err: 4.1e-06

### (b) Write a function, boot_fn(), that takes as input the Default data set as well as an index of the observations, and that outputs the coefficient estimates for income and balance in the multiple logistic regression model. 

In [101]:
def boot_fn(default, idx):
    # Subset the data based on the provided indices
    boot_sample = default.iloc[idx]
    
    # Fit logistic regression model to the bootstrap sample
    mod = smf.glm(formula='default ~ income + balance', data=boot_sample, family=sm.families.Binomial()).fit()
    
    # Extract coefficient estimates for 'income' and 'balance'
    coef_income = mod.params[1]
    coef_balance = mod.params[2]
    
    return [coef_income, coef_balance]

### (c) Following the bootstrap example in the lab, use your boot_fn() function to estimate the standard errors of the logistic regression coefficients for income and balance. 

In [106]:
def boot_SE(func, D, n=None, B=1000, seed=0):
    rng = np.random.default_rng(seed)
    first_, second_ = 0, 0
    n = n or D.shape[0]
    
    for _ in range(B):
        idx = rng.choice(n, replace=True)
        coef_income, coef_balance = func(D, idx)  # Unpack coefficients from the list
        first_ += coef_income
        second_ += coef_income ** 2
        
    return np.sqrt(second_ / B - (first_ / B) ** 2)

In [109]:
estimated_error = boot_SE(
    boot_fn ,             
    default ,              
    B=1000,            
    seed=0
)

estimated_error

0.0004641062613835811

### (d) Comment on the estimated standard errors obtained using the sm.GLM() function and using the bootstrap.

- The standard error is larger when using th bootstrap method?

## 9. We will now consider the Boston housing data set, from the ISLP library.


In [118]:
boston = load_data('Boston')

#### (a) Based on this data set, provide an estimate for the population mean of medv. Call this estimate µ


In [122]:
µ = boston['medv'].mean()

In [120]:
µ

22.532806324110698

### (b) Provide an estimate of the standard error of µˆ. Interpret this result. Hint: We can compute the standard error of the sample mean by dividing the sample standard deviation by the square root of the number of observations.


In [124]:
std = boston['medv'].std()
se_µ = std / np.sqrt(len(boston))
se_µ

0.4088611474975351

- On average, the sample mean estimate of the median house value differes form the true population mean by approx .4089

### (c) Now estimate the standard error of µˆ using the bootstrap. How does this compare to your answer from (b)?


In [128]:
def boot_fn(data, idx):
    # Subset the data based on the provided indices
    boot_sample = data.iloc[idx]
    
    # Compute the sample mean of 'medv' for the bootstrap sample
    sample_mean = boot_sample['medv'].mean()
    
    return sample_mean

# Function to compute standard error using bootstrap
def boot_SE(func, data, B=1000, seed=0):
    rng = np.random.default_rng(seed)
    sample_means = []
    
    for _ in range(B):
        idx = rng.choice(data.index, size=len(data), replace=True)
        sample_mean = func(data, idx)
        sample_means.append(sample_mean)
    
    # Compute standard error
    se = np.std(sample_means)
    
    return se

# Estimate standard error using bootstrap
se_bootstrap = boot_SE(boot_fn, boston)
print("Standard Error using Bootstrap:", se_bootstrap)

Standard Error using Bootstrap: 0.4125347675099619


- they are almost identical. Bootstrap standard error is slightly higher

### (d) Based on your bootstrap estimate from (c), provide a 95 % confidence interval for the mean of medv. Compare it to the results obtained by using Boston['medv'].std() and the two standard error rule (3.9). Hint: You can approximate a 95 % confidence interval using the formula [ˆ µ − 2SE(ˆ µ), µˆ + 2SE(ˆ µ)].


In [165]:
mu_hat   = np.mean(boston['medv'])
conf_low = mu_hat - (2*se_bootstrap)
conf_hi  = mu_hat + (2*se_bootstrap)

pd.Series({'mu': mu_hat, 
           'SE': se_bootstrap,
           '[0.025': conf_low,
           '0.975]': conf_hi})

mu        22.532806
SE         0.412535
[0.025    21.707737
0.975]    23.357876
dtype: float64

### (e) Based on this data set, provide an estimate, µˆmed, for the median value of medv in the population.


In [167]:
median_hat = np.median(boston['medv'])
median_hat

21.2

### (f) We now would like to estimate the standard error of µˆmed. Unfortunately, there is no simple formula for computing the standard error of the median. Instead, estimate the standard error of the median using the bootstrap. Comment on your findings.


In [168]:
def boot_fn(data, idx):
    # Subset the data based on the provided indices
    boot_sample = data.iloc[idx]
    
    # Compute the sample mean of 'medv' for the bootstrap sample
    sample_median = boot_sample['medv'].median()
    
    return sample_median

In [169]:
# Function to compute standard error using bootstrap
def boot_SE(func, data, B=1000, seed=0):
    rng = np.random.default_rng(seed)
    sample_medians = []
    
    for _ in range(B):
        idx = rng.choice(data.index, size=len(data), replace=True)
        sample_median = func(data, idx)
        sample_medians.append(sample_median)
    
    # Compute standard error
    se = np.std(sample_medians)
    
    return se

# Estimate standard error using bootstrap
se_bootstrap = boot_SE(boot_fn, boston)
print("Standard Error using Bootstrap:", se_bootstrap)

Standard Error using Bootstrap: 0.3694462207141924


### (g) Based on this data set, provide an estimate for the tenth percentile of medv in Boston census tracts. Call this quantity µˆ0.1. (You can use the np.percentile() function.)



In [170]:
tenth_percentile = np.percentile(boston['medv'], 10)
tenth_percentile

12.75

### (h) Use the bootstrap to estimate the standard error of µˆ0.1. Comment on your findings

In [171]:
def boot_fn(data, idx):
    # Subset the data based on the provided indices
    boot_sample = data.iloc[idx]
    
    # Compute the sample mean of 'medv' for the bootstrap sample
    sample_tenth_percent = np.percentile(boot_sample['medv'], 10)
    
    return sample_tenth_percent

In [172]:
se_bootstrap = boot_SE(boot_fn, boston)
print("Standard Error µˆ0.1 Bootstrap:", se_bootstrap)

Standard Error µˆ0.1 Bootstrap: 0.5034541091301172
